In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Input, Dense, Lambda
from keras.optimizers import Adam
from keras import backend as K
from google.colab import drive
drive.mount('/content/drive')



ModuleNotFoundError: No module named 'cv2'

In [3]:
!pip install opencv-python


In [ ]:
def load_and_preprocess_images(folder_path, img_size=(100, 100), num_images=None, random_seed=None):
    images = []
    image_files = [filename for filename in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, filename)) and
                   filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

    if num_images is not None:
        if random_seed is not None:
            np.random.seed(random_seed)
        selected_files = np.random.choice(image_files, num_images, replace=False)
    else:
        selected_files = image_files

    for filename in selected_files:
        img_path = os.path.join(folder_path, filename)

        try:
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, img_size)
                images.append(img.flatten())
            else:
                print(f"Error loading {img_path}. Skipping this image.")
        except Exception as e:
            print(f"Error processing {img_path}: {str(e)}. Skipping this image.")

    return np.array(images)

def apply_pca(images, n_components=50):
    pca = PCA(n_components=n_components)
    pca_result = pca.fit_transform(images)
    reconstructed_images = pca.inverse_transform(pca_result)
    return pca_result, reconstructed_images, pca.components_

def display_images(original_images, reconstructed_images, img_size=(100, 100), num_images=5):
    plt.figure(figsize=(15, 6))
    for i in range(num_images):
        plt.subplot(2, num_images, i + 1)
        plt.imshow(original_images[i].reshape(img_size), cmap='gray')
        plt.title('Original')
        plt.axis('off')

        plt.subplot(2, num_images, i + 1 + num_images)
        plt.imshow(reconstructed_images[i].reshape(img_size), cmap='gray')
        plt.title('Reconstructed (PCA)')
        plt.axis('off')

    plt.tight_layout()
    plt.show()

def display_pca_components(components, img_size=(100, 100), num_components=3):
    plt.figure(figsize=(15, 4))
    for i in range(num_components):
        plt.subplot(1, num_components, i + 1)
        plt.imshow(components[i].reshape(img_size), cmap='gray')
        plt.title(f'Principal Component {i+1}')
        plt.axis('off')

    plt.tight_layout()
    plt.show()

def display_images2(original_images, reconstructed_images1, reconstructed_images2, img_size=(100, 100), num_images=5):
    plt.figure(figsize=(15, 6))
    for i in range(num_images):
        # Display original images
        plt.subplot(3, num_images, i + 1)
        plt.imshow(original_images[i].reshape(img_size), cmap='gray')
        plt.title('Original')
        plt.axis('off')

        # Display Autoencoder reconstructed images
        plt.subplot(3, num_images, i + 1 + num_images)
        plt.imshow(reconstructed_images1[i].reshape(img_size), cmap='gray')
        plt.title('Re AE')
        plt.axis('off')

        # Display VAE reconstructed images
        plt.subplot(3, num_images, i + 1 + 2 * num_images)
        plt.imshow(reconstructed_images2[i].reshape(img_size), cmap='gray')
        plt.title('Re VAE')
        plt.axis('off')

    plt.tight_layout()
    plt.show()


def build_autoencoder(input_size, encoding_dim):
    input_img = Input(shape=(input_size,))

    encoded = Dense(256, activation='relu')(input_img)
    encoded = Dense(128, activation='relu')(encoded)
    encoded = Dense(64, activation='relu')(encoded)
    encoded = Dense(32, activation='relu')(encoded)
    encoded = Dense(16, activation='relu')(encoded)
    encoded = Dense(8, activation='relu')(encoded)
    encoded = Dense(encoding_dim, activation='relu')(encoded)

    decoded = Dense(8, activation='relu')(encoded)
    decoded = Dense(16, activation='relu')(decoded)
    decoded = Dense(32, activation='relu')(decoded)
    decoded = Dense(64, activation='relu')(decoded)
    decoded = Dense(128, activation='relu')(decoded)
    decoded = Dense(256, activation='relu')(decoded)
    decoded = Dense(input_size, activation='sigmoid')(decoded)

    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return autoencoder

def train_autoencoder(images, encoding_dim=10, epochs=50, batch_size=32):
    input_size = images.shape[1]
    autoencoder = build_autoencoder(input_size, encoding_dim)
    scaler = MinMaxScaler()
    images_scaled = scaler.fit_transform(images)
    x_train, x_test = train_test_split(images_scaled, test_size=0.2, random_state=42)
    autoencoder.fit(x_train, x_train, epochs=epochs, batch_size=batch_size, shuffle=True, validation_data=(x_test, x_test))
    return autoencoder

def apply_autoencoder(autoencoder, images):
    scaler = MinMaxScaler()
    images_scaled = scaler.fit_transform(images)
    reconstructed_images = autoencoder.predict(images_scaled)
    reconstructed_images = scaler.inverse_transform(reconstructed_images)
    return reconstructed_images


def build_vae(input_size, encoding_dim):
    input_img = Input(shape=(input_size,))
    h = Dense(256, activation='relu')(input_img)
    z_mean = Dense(encoding_dim)(h)
    z_log_var = Dense(encoding_dim)(h)

    def sampling(args):
        z_mean, z_log_var = args
        batch = K.shape(z_mean)[0]
        dim = K.int_shape(z_mean)[1]
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + K.exp(0.5 * z_log_var) * epsilon

    z = Lambda(sampling, output_shape=(encoding_dim,))([z_mean, z_log_var])

    decoder_h = Dense(256, activation='relu')
    decoder_mean = Dense(input_size, activation='sigmoid')
    h_decoded = decoder_h(z)
    x_decoded_mean = decoder_mean(h_decoded)

    vae = Model(input_img, x_decoded_mean)

    xent_loss = input_size * K.mean(K.binary_crossentropy(input_img, x_decoded_mean))
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    vae_loss = xent_loss + kl_loss

    vae.add_loss(vae_loss)
    vae.compile(optimizer=Adam(learning_rate=0.001))
    return vae

def train_vae(images, encoding_dim=10, epochs=50, batch_size=32):
    input_size = images.shape[1]
    vae = build_vae(input_size, encoding_dim)
    scaler = MinMaxScaler()
    images_scaled = scaler.fit_transform(images)
    x_train, x_test = train_test_split(images_scaled, test_size=0.2, random_state=42)
    vae.fit(x_train, epochs=epochs, batch_size=batch_size, shuffle=True, validation_data=(x_test, None))
    return vae

def apply_vae(vae, images):
    scaler = MinMaxScaler()
    images_scaled = scaler.fit_transform(images)
    reconstructed_images = vae.predict(images_scaled)
    reconstructed_images = scaler.inverse_transform(reconstructed_images)
    return reconstructed_images



In [ ]:
training_folder_path = "/content/drive/MyDrive/MUST/KU ppt/DataA2"
example_folder_path = "/content/drive/MyDrive/MUST/KU ppt/DataA"

training_images = load_and_preprocess_images(training_folder_path, num_images=None, random_seed=random_seed)

n_components = 5
pca_result_train, pca_reconstructed_train_images, pca_components_train = apply_pca(training_images, n_components)

example_images = load_and_preprocess_images(example_folder_path, num_images=None, random_seed=random_seed)

num_images_to_display = 10


autoencoder = train_autoencoder(training_images, encoding_dim=10, epochs=50, batch_size=32)
autoencoder_reconstructed_images = apply_autoencoder(autoencoder, training_images)

display_images(example_images, autoencoder_reconstructed_images, num_images=num_images_to_display)


vae = train_vae(training_images, encoding_dim=10, epochs=50, batch_size=32)
vae_reconstructed_images = apply_vae(vae, training_images)


display_images(example_images, vae_reconstructed_images, num_images=num_images_to_display)
display_images2(example_images, autoencoder_reconstructed_images, vae_reconstructed_images, num_images=num_images_to_display)